In [79]:
site = 'UKY'

import os
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
import json
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, ComponentModelType, ServiceType
import time
import paramiko
import re

credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
slice_manager = SliceManager(oc_host=orchestrator_host, cm_host=credmgr_host, project_name='all', scope='all')
slice_manager.initialize()

t = ExperimentTopology()
cap = Capacities()
cap.set_fields(core=32, ram=128, disk=10)
n1 = t.add_node(name='n1', site=site)
n1.set_properties(capacities=cap, image_type='qcow2', image_ref='default_ubuntu_20')
n2 = t.add_node(name='n2', site=site)
n2.set_properties(capacities=cap, image_type='qcow2', image_ref='default_ubuntu_20')
n1.add_component(model_type=ComponentModelType.SmartNIC_ConnectX_6, name='n1-nic1')
n2.add_component(model_type=ComponentModelType.SmartNIC_ConnectX_6, name='n2-nic1')
t.add_network_service(name='bridge1', nstype=ServiceType.L2Bridge, interfaces=[n1.interface_list[0], n2.interface_list[0]])
slice_graph = t.serialize()
ssh_key = None
with open ("/home/fabric/.ssh/id_rsa.pub", "r") as myfile:
    ssh_key=myfile.read()
    ssh_key=ssh_key.strip()
status, reservations = slice_manager.create(slice_name="test_harness_latency_mtu_bandwidth", slice_graph=slice_graph, ssh_key=ssh_key)
if(status != Status.OK):
    print(status)
    print(reservations)
    raise Exception("Slice creation failed. One thing to do: try renaming it?")
slice_id=reservations[0].slice_id

seconds_to_sleep = 15.0
while True:
    return_status, slices = slice_manager.slices(excludes=[SliceState.Dead])
    if(list(filter(lambda x : x.slice_id == slice_id, slices))[0].slice_state == "StableOK"):
        print("Slice is StableOK.")
        break
    elif(list(filter(lambda x : x.slice_id == slice_id, slices))[0].slice_state != "Configuring"):
        print("Slice state: " + list(filter(lambda x : x.slice_id == slice_id, slices))[0].slice_state)
#         print(slice_manager.slivers(slice_id=slice_id))
        slice_object = list(filter(lambda x: x.slice_id == slice_id, slices))[0]
        status, slivers = slice_manager.slivers(slice_object=slice_object)
        for s in slivers:
            status, sliver_status = slice_manager.sliver_status(sliver=s)
            print("Response Status {}".format(status))
            print("Sliver Status {}".format(sliver_status))
        raise Exception("Slice creation failed.")
    else:
        print("Slice is configuring. Trying again in " + str(seconds_to_sleep) + " seconds.")
    time.sleep(seconds_to_sleep)
    
slice_object = list(filter(lambda x: x.slice_id == slice_id, slices))[0]
status, slivers = slice_manager.slivers(slice_object=slice_object)
n1_ip = list(filter(lambda sliver : sliver.name == "n1",slivers))[0].management_ip
n2_ip = list(filter(lambda sliver : sliver.name == "n2",slivers))[0].management_ip
print(n1_ip)
print(n2_ip)
key = paramiko.RSAKey.from_private_key_file("/home/fabric/.ssh/id_rsa")
clientn1 = paramiko.SSHClient()
clientn1.load_system_host_keys()
clientn1.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())
clientn1.set_missing_host_key_policy(paramiko.AutoAddPolicy())
connection_attempts = 0
while True:
    try:
        clientn1.connect(n1_ip,username='ubuntu',pkey = key)
        break
    except Exception as exception:
        print(exception)
        connection_attempts += 1
        if(connection_attempts < 10):
            print("Connection failed. Will try to connect again in a few seconds.")
            time.sleep(5)
        else:
            raise Exception("Connection to server failed.")
clientn2 = paramiko.SSHClient()
clientn2.load_system_host_keys()
clientn2.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())
clientn2.set_missing_host_key_policy(paramiko.AutoAddPolicy())
connection_attempts = 0
while True:
    try:
        clientn2.connect(n2_ip,username='ubuntu',pkey = key)
        break
    except Exception as exception:
        print(exception)
        connection_attempts += 1
        if(connection_attempts < 10):
            print("Connection failed. Will try to connect again in a few seconds.")
            time.sleep(5)
        else:
            raise Exception("Connection to server failed.")
def get_interface_before_last(stdout):
    interface = re.findall(r"[0-9]: [A-Za-z][A-Za-z][A-Za-z][0-9]", stdout.read().decode("utf-8"))
    interface = interface[-2:-1]
    interface = re.findall("[A-Za-z][A-Za-z][A-Za-z][0-9]", interface[0])
    return interface[0]

stdin, stdout, stderr = clientn1.exec_command('sudo apt-get update && sudo apt-get install -y iperf iperf3')
stdin, stdout, stderr = clientn2.exec_command('sudo apt-get update && sudo apt-get install -y iperf iperf3')

################################Setting up the IP addresses and activating the interfaces
stdin, stdout, stderr = clientn1.exec_command('ip a')
interface_n1 = get_interface_before_last(stdout)
ip_of_interface_on_n1 = "192.168.10.51"
stdin, stdout, stderr = clientn1.exec_command('sudo ip addr add ' + ip_of_interface_on_n1 + '/24 dev ' + interface_n1)
stdin, stdout, stderr = clientn1.exec_command('sudo ip link set dev ' + interface_n1 + ' up mtu 9000')
stdin, stdout, stderr = clientn2.exec_command('ip a')
interface_n2 = get_interface_before_last(stdout)
ip_of_interface_on_n2 = "192.168.10.52"
stdin, stdout, stderr = clientn2.exec_command('sudo ip addr add ' + ip_of_interface_on_n2 + '/24 dev ' + interface_n2)
stdin, stdout, stderr = clientn2.exec_command('sudo ip link set dev ' + interface_n2 + ' up mtu 9000')

################################Latency
stdin, stdout, stderr = clientn1.exec_command('ping -c 5 ' + ip_of_interface_on_n2 + ' | grep rtt')
output1 = stdout.read().decode("utf-8")
stdin, stdout, stderr = clientn2.exec_command('ping -c 5 ' + ip_of_interface_on_n1 + ' | grep rtt')
output1 += "\n" + stdout.read().decode("utf-8")

################################MTU
output2 = ""
ping_packets_count = 3
ping_packet_sizes = [9000, 8950, 8000, 1500, 1450, 1400, 1000, 500, 100, 50]
for ping_packet_size in ping_packet_sizes:
    stdin, stdout, stderr = clientn1.exec_command('ping -M do -s ' + str(ping_packet_size) + ' -c ' + str(ping_packets_count) + ' ' + ip_of_interface_on_n2)
    ping_string = stdout.read().decode("utf-8")
#     print(ping_string)
    ping_string = re.findall("[0-9] received", ping_string)
    ping_string = re.findall("[0-9]", ping_string[0])
    if(int(ping_string[0]) == ping_packets_count):
        out = "Packet size " + str(ping_packet_size + 8) + " is enabled."
        output2 += out
        break
    else:
        print("Packet " + str(ping_packet_size + 8) + " too large.")
for ping_packet_size in ping_packet_sizes:
    stdin, stdout, stderr = clientn2.exec_command('ping -M do -s ' + str(ping_packet_size) + ' -c ' + str(ping_packets_count) + ' ' + ip_of_interface_on_n1)
    ping_string = stdout.read().decode("utf-8")
#     print(ping_string)
    ping_string = re.findall("[0-9] received", ping_string)
    ping_string = re.findall("[0-9]", ping_string[0])
    if(int(ping_string[0]) == ping_packets_count):
        out = "Packet size " + str(ping_packet_size + 8) + " is enabled."
        output2 += "\n" + out
        break
    else:
        print("Packet " + str(ping_packet_size + 8) + " too large.")
        
################################Bandwidth
stdin, stdout, stderr = clientn1.exec_command('iperf -s > /dev/null 2>&1 &')
output3 = ""
stdin, stdout, stderr = clientn2.exec_command('iperf -c ' + ip_of_interface_on_n1 + ' -P 32 -w 32M')
iperf_string = stdout.read().decode("utf-8")
iperf_string2 = re.findall("........../sec", iperf_string)
if(len(iperf_string2) > 0):
    output3 = iperf_string2[-1]
else:
    output3 = iperf_string

################################Printing
print("#####Information about latency with ping: \n" + output1)
print("#####Information about mtu with ping: \n" + output2)
print("#####Information about bandwidth with iperf: \n" + output3)

slice_manager.delete(slice_object=slice_object)

Slice is configuring. Trying again in 15.0 seconds.
Slice is configuring. Trying again in 15.0 seconds.
Slice is configuring. Trying again in 15.0 seconds.
Slice is configuring. Trying again in 15.0 seconds.
Slice is configuring. Trying again in 15.0 seconds.
Slice is configuring. Trying again in 15.0 seconds.
Slice is configuring. Trying again in 15.0 seconds.
Slice is configuring. Trying again in 15.0 seconds.
Slice is StableOK.
128.163.179.38
128.163.179.39
Packet 9008 too large.
Packet 9008 too large.
#####Information about latency with ping: 
rtt min/avg/max/mdev = 0.078/202.947/1013.974/405.513 ms, pipe 2

rtt min/avg/max/mdev = 0.074/0.104/0.147/0.028 ms

#####Information about mtu with ping: 
Packet size 8958 is enabled.
Packet size 8958 is enabled.
#####Information about bandwidth with iperf: 
98.1 Gbits/sec


(<Status.OK: 1>, None)